In [ ]:
import glob
#긍정리뷰 파일들을 로딩
pos_review=(glob.glob("d:/data/imdb/train/pos/*.txt"))
#print(pos_review)
lines_pos=[]
for i in pos_review:
    try:
        f=open(i, "r") #읽기 모드로 파일 오픈
        temp=f.readlines()[0] 
        lines_pos.append(temp)
        f.close()
    except Exception as e:
        continue
        
print(len(lines_pos)) #12490건의 긍정 리뷰
print(lines_pos[:1])

In [ ]:
import glob
#긍정리뷰 파일들을 로딩
neg_review=(glob.glob("d:/data/imdb/train/neg/*.txt"))
#print(pos_review)
lines_neg=[]
for i in neg_review:
    try:
        f=open(i, "r") #읽기 모드로 파일 오픈
        temp=f.readlines()[0] 
        lines_neg.append(temp)
        f.close()
    except Exception as e:
        continue
        
print(len(lines_neg)) #12489건의 부정 리뷰
print(lines_neg[:1])

In [ ]:
#긍정,부정 리뷰를 합침
total_text=lines_pos + lines_neg
len(total_text)

In [ ]:
import numpy as np
from nltk.corpus import stopwords
#긍정,부정 클래스에 라벨링
x= np.array(["pos","neg"])
class_Index=np.repeat(x, [len(lines_pos), len(lines_neg)],
                     axis=0)
print(len(class_Index))
print(class_Index)
stop_words=stopwords.words("english") #영어 불용어 사전


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#불용어를 제거, 단어들에 Tfidf 가중치를 부여한 후 
#문서-단어 행렬로 변환하는 코드
vect=TfidfVectorizer(stop_words=stop_words).fit(total_text)
X_train_vectorized=vect.transform(total_text)
X_train_vectorized.index=class_Index
print(X_train_vectorized[:1])
print(class_Index[:1])

In [ ]:
#로지스틱 회귀분석
from sklearn.linear_model import LogisticRegression
logit=LogisticRegression(random_state=10)
logit.fit(X_train_vectorized, class_Index)

In [ ]:
#긍정 리뷰의 예측 정확도를 계산하는 함수
def pos_review(model):
    count_all=0 #전체 카운트 변수
    count=0 # 긍정 리뷰를 카운트하는 변수
    num=10 # 실험횟수
    tests1=[]
    for idx in range(0,num):
        # 0~99 인덱스에 해당하는 리뷰 문서를 불러옴
        pos_review_test=(glob.glob("d:/data/imdb/test/pos/*.txt"))[idx]
        f=open(pos_review_test,"r",encoding="utf-8")
        tests1.append(f.readlines())
        f.close()
        
        for test in tests1:
            preds=model.predict(vect.transform(test)) #예측값 계산
            result=preds[0]
            #print(result)
            if result=="pos": #긍정으로 예측했으면 카운트 증가 처리
                count+=1
            count_all += 1
    rate=count*100 / count_all
    print("예측 정확도:{0:.1f}%".format(rate))
    
pos_review(logit)  #함수 호출  

In [ ]:
#부정 리뷰의 예측 정확도를 계산하는 함수
def neg_review(model):
    count_all=0 #전체 카운트 변수
    count=0 # 부정 리뷰를 카운트하는 변수
    num=10 # 실험횟수
    tests2=[]
    for idx in range(0,num):
        # 0~99 인덱스에 해당하는 리뷰 문서를 불러옴
        neg_review_test=(glob.glob("d:/data/imdb/test/neg/*.txt"))[idx]
        f=open(neg_review_test,"r",encoding="utf-8")
        tests2.append(f.readlines())
        f.close()
        
        for test in tests2:
            preds=model.predict(vect.transform(test)) #예측값 계산
            result=preds[0]
            #print(result)
            if result=="neg": #부정으로 예측했으면 카운트 증가 처리
                count+=1
            count_all += 1
    rate=count*100 / count_all
    print("예측 정확도:{0:.1f}%".format(rate))
    
neg_review(logit)  #함수 호출  

In [ ]:
#의사결정나무 모형
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(random_state=10)
tree.fit(X_train_vectorized, class_Index)

In [ ]:
pos_review(tree) #학습용
neg_review(tree) #검증용

In [ ]:
#랜덤포레스트 모형
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=100,random_state=10) #트리갯수 100개
forest.fit(X_train_vectorized, class_Index)

In [ ]:
pos_review(forest) #학습용
neg_review(forest) #검증용

In [ ]:
#KNN 모형
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=2) #2개의 이웃
knn.fit(X_train_vectorized, class_Index)

In [ ]:
pos_review(knn)
neg_review(knn)

In [ ]:
#인공신경망
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(random_state=10)
mlp.fit(X_train_vectorized, class_Index)

In [ ]:
pos_review(mlp)
neg_review(mlp)

In [ ]:
#SVM
from sklearn.svm import SVC
svm=SVC(random_state=10)
svm.fit(X_train_vectorized, class_Index)

In [ ]:
pos_review(svm)
neg_review(svm)